In [1]:
import pandas as pd
import re

In [2]:
pattern = re.compile(r"\[(.*?)\]")

wake_up = []
fall_asleep = []
begin_shift = []

with open("input.txt") as f:
    guards = []
    for l in f:
        a, b = pattern.match(l).span()
        date = "2018"+l[a+1: b-1][4:]
        event = l[b+1:].strip()
        if event == "wakes up":
            wake_up.append((date, 1))
        elif event == "falls asleep":
            fall_asleep.append((date, 1))
        else:
            guard_id = int(event.split()[1][1:])
            begin_shift.append((date, guard_id))

In [3]:
df = pd.concat([pd.DataFrame(begin_shift, columns=["date", "guard"]),
               pd.DataFrame(fall_asleep, columns=["date", "fall_asleep"]),
               pd.DataFrame(wake_up, columns=["date", "wake_up"])], sort=False)

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").set_index("date")
df["guard"] = df["guard"].ffill()

In [4]:
results = []
for guard_id, grp in df.groupby("guard"):
    grp = grp.reindex(pd.date_range(grp.index[0], grp.index[~0], freq="60S")).fillna(0).query("index.dt.hour == 0")
    grp["sleeping"] = grp["fall_asleep"].cumsum()-grp["wake_up"].cumsum()
    grp["min"] = grp.index.minute
    r = grp.groupby("min")["sleeping"].sum()
    total_minutes_asleep = r.sum()
    k = r.sort_values(ascending=False)
    d = {"guard_id": int(guard_id), "total_minutes_asleep": int(total_minutes_asleep), 
         "most_freq_minute": k.index[0], "count_most_freq_minute": int(k.iloc[0])}
    results.append(d)

In [5]:
res = pd.DataFrame(results, columns=["guard_id", "total_minutes_asleep", "most_freq_minute", "count_most_freq_minute"])

In [6]:
res

,guard_id,total_minutes_asleep,most_freq_minute,count_most_freq_minute
0,103,283,38,11
1,109,422,24,12
2,239,227,47,9
3,277,303,35,8
4,281,0,59,0
5,433,284,44,10
6,677,231,36,9
7,727,449,49,13
8,739,280,37,11
9,821,0,59,0


# Part 1

In [7]:
res.sort_values("total_minutes_asleep", ascending=False).head(1)

,guard_id,total_minutes_asleep,most_freq_minute,count_most_freq_minute
7,727,449,49,13


In [8]:
r = res.sort_values("total_minutes_asleep", ascending=False)
r.iloc[0]["guard_id"] *r.iloc[0]["most_freq_minute"]

35623

# Part 2

In [9]:
res.sort_values("count_most_freq_minute", ascending=False).head(1)

,guard_id,total_minutes_asleep,most_freq_minute,count_most_freq_minute
11,1097,409,21,15


In [10]:
r = res.sort_values("count_most_freq_minute", ascending=False)
r.iloc[0]["guard_id"] *r.iloc[0]["most_freq_minute"]

23037